In [2]:
import os
import re
import time

from dotenv import load_dotenv
import google.generativeai as genai
import google.generativeai.types as types
from huggingface_hub import login
import pandas as pd

SAMPLES_PER_CALL = 100
NUM_CALLS_TRANSLATION = 200
SECONDS_TO_SLEEP = 6 

In [3]:
load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=API_KEY)

In [4]:
model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
# save_translation_pairs_declaration = {
#     "name": "save_translation_pairs",
#     "description": f"Lưu một lô (batch) gồm {SAMPLES_PER_CALL} cặp câu dịch Anh-Việt.",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "pairs": {
#                 "type": "array",
#                 "description": f"Một danh sách (list) gồm {SAMPLES_PER_CALL} đối tượng (object) cặp câu dịch.",
#                 "items": {
#                     "type": "object",
#                     "properties": {
#                         "English": {
#                             "type": "string",
#                             "description": "Câu đầy đủ bằng tiếng Anh."
#                         },
#                         "Vietnamese": {
#                             "type": "string",
#                             "description": "Bản dịch tiếng Việt chính xác của câu đó."
#                         }
#                     },
#                     "required": ["English", "Vietnamese"]
#                 }
#             }
#         },
#         "required": ["pairs"]
#     }
# }

In [ ]:
# gemini_tools = [types.Tool(function_declarations=[save_translation_pairs_declaration])]

In [ ]:
# def generate_data(prompt_template, num_calls,
#                   output_filename="synthetic_translation_data.csv",
#                   model_obj=None, tools=None,
#                   max_retries=2):

#     if model_obj is None:
#         model_obj = model
#     if tools is None:
#         tools = gemini_tools

#     if os.path.exists(output_filename):
#         try:
#             all_samples_df = pd.read_csv(output_filename)
#             print(f"loaded {len(all_samples_df)} from '{output_filename}'")
#         except Exception:
#             all_samples_df = pd.DataFrame(columns=["English", "Vietnamese"])
#     else:
#         all_samples_df = pd.DataFrame(columns=["English", "Vietnamese"])

#     for call in range(num_calls):
#         print(f"call {call+1}/{num_calls}...")

#         for attempt in range(1, max_retries + 2):
#             try:
#                 response = model_obj.generate_content(prompt_template, tools=tools)

#                 if not response.candidates or len(response.candidates) == 0:
#                     raise ValueError("No candidates (blocked by safety)")

#                 if not response.candidates[0].content.parts or len(response.candidates[0].content.parts) == 0:
#                     raise ValueError("No parts in response")

#                 part = response.candidates[0].content.parts[0]

#                 if not part.function_call:
#                     raise ValueError("No function_call")

#                 if part.function_call.name != "save_translation_pairs":
#                     raise ValueError("Wrong function name")

#                 pairs = part.function_call.args.get("pairs")
#                 if not pairs:
#                     raise ValueError("Missing pairs")

#                 new_df = pd.DataFrame(pairs, columns=["English", "Vietnamese"])
#                 all_samples_df = pd.concat([all_samples_df, new_df], ignore_index=True)
                
#                 print(f"{len(pairs)} pairs added, total {len(all_samples_df)})")

#                 all_samples_df.to_csv(output_filename, index=False, encoding="utf-8")
#                 print(f"saved to {output_filename}")

#                 break

#             except Exception as e:
#                 print(f"error at attempt {attempt}: {e}")

#                 if attempt < max_retries + 1:
#                     time.sleep(1)

#         print(f"{SECONDS_TO_SLEEP}s cool down")
#         time.sleep(SECONDS_TO_SLEEP)

#     return all_samples_df

In [8]:
PROMPT_TRANSLATION = f"""
Nhiệm vụ của bạn là tạo dữ liệu cho dự án nghiên cứu dịch máy của tôi.
Hãy tạo ra chính xác {SAMPLES_PER_CALL} cặp câu dịch Anh-Việt mới, đa dạng về chủ đề đời sống hàng ngày (ví dụ: chào hỏi, ăn uống, thời tiết, du lịch...).
"""

In [ ]:
PROMPT_TRANSLATION_V2 = f"""
Nhiệm vụ của bạn là tạo dữ liệu cho dự án nghiên cứu dịch máy của tôi.
Hãy tạo ra chính xác {SAMPLES_PER_CALL} cặp câu dịch Anh-Việt mới, đa dạng về chủ đề đời sống hàng ngày (ví dụ: chào hỏi, ăn uống, thời tiết, du lịch...).

ĐỊNH DẠNG ĐẦU RA:
Mỗi cặp câu theo format: English sentence|||Vietnamese translation
Mỗi cặp trên một dòng riêng.
Không đánh số, không gạch đầu dòng.
"""

def generate_translation_data(prompt_template, num_calls, output_filename="synthetic_translation_data_V2.csv"):
    all_samples = []
    
    if os.path.exists(output_filename):
        try:
            existing_df = pd.read_csv(output_filename)
            all_samples = existing_df[['English', 'Vietnamese']].to_dict('records')
            print(f"loaded {len(all_samples)} from '{output_filename}'")
        except Exception as e:
            print(f"error reading file: {e}")
    
    for i in range(num_calls):
        print(f"call {i+1}/{num_calls}...")
        
        try:
            response = model.generate_content(prompt_template)
            text_data = response.text
            
            lines = [line.strip() for line in text_data.split('\n') if line.strip() and '|||' in line]
            
            for line in lines:
                parts = line.split('|||')
                if len(parts) == 2:
                    all_samples.append({'English': parts[0].strip(), 'Vietnamese': parts[1].strip()})
            
            print(f"{len(lines)} pairs added, total {len(all_samples)})")
            
            df_temp = pd.DataFrame(all_samples)
            df_temp.to_csv(output_filename, index=False, encoding='utf-8')
            print(f"saved to {output_filename}")

        except Exception as e:
            print(f"error at call {i+1}: {e}")
            
        finally:
            print(f"{SECONDS_TO_SLEEP}s cool down")
            time.sleep(SECONDS_TO_SLEEP)
            
    return pd.DataFrame(all_samples)

In [6]:
output_filename = "synthetic_translation_data_V2.csv"
translation_df = generate_translation_data(PROMPT_TRANSLATION_V2, NUM_CALLS_TRANSLATION, output_filename=output_filename)

loaded 20331 from 'synthetic_translation_data_V2.csv'
call 1/200...
6s cool down


KeyboardInterrupt: 